In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from helpers import utils, pipelines, models

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

import dice_ml

import multiprocessing

from collections.abc import Iterable
from functools import partial

from copy import deepcopy

from data_augmentaion.data_augmentator import DataAugmentor 

import json

### Define research parameters

In [3]:
regression_task = False
continuous_features=['Age', 'Credit amount', 'Duration']
metric = 'f1'
test_size_proportion = 0.33
augment_sample = 0.33

# params_search = {
#     'metrics': ['f1', 'roc_auc', 'accuracy'],
#     'test_proportions': [0.33],
#     'augment_sample': [0.2, 0.5, 1.0],
#     'augment_methods': ['cf_random', 'cf_genetic', 'cf_kdtree', 'random', 'smote'],
#     # 'cf_genetic__proximity_weight': [0.2, 5],
#     # 'cf_genetic__diversity_weight': [0.2, 5],
# }

# settings = utils.get_settings_options(params_search)
# for k in settings:
#     print(k, len(settings[k]))

settings = [
    {'method': 'random'},
    {'method': 'smote'},
    {'method': 'cf_random'},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 0.2}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 1}},
    {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 0.2}},
    {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 1}},
]

In [4]:
search_pipelines = pipelines.get_classification_pipelines()
search_parameters = models.parameters

### Read and preprocess data

In [5]:
raw_data = pd.read_csv(r"datasets\german_credit.csv", index_col=0)
df = utils.preprocess_german(raw_data)
target = 'Risk'

#### split data

In [6]:
# all data
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_proportion, random_state=42)

In [7]:
display(y_train.value_counts())

1    471
0    199
Name: Risk, dtype: int64

In [8]:
full_results_df = pd.DataFrame()

### Whole dataset scores

In [9]:
print(f'whole adult dataset {metric} scores:')
whole_best_ests, whole_scores = utils.fit_and_evaluate(X_train, y_train, X_test, y_test,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
whole_dataset_result_df = pd.DataFrame.from_dict(whole_scores, orient='index', columns=[f'full data {metric} score'])
whole_dataset_result_df.T

whole adult dataset f1 scores:


,lg,rf,xgb
full data f1 score,0.823529,0.822669,0.815109


In [10]:
full_results_df = pd.concat([full_results_df, whole_dataset_result_df], axis=1)

### running experiments

In [11]:
best_estimators = {}
best_scores = {}
for i, s in enumerate(settings):
    print(f'{i} / {len(settings)}, {s}')
    augmentor = DataAugmentor(X_train, y_train, X_test, y_test,
                             method=s['method'], regression=regression_task,
                             continuous_feats=continuous_features,
                             kw_args=s.get('kw_args', {})
                             )
    X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
    X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
    
    best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
        utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test, y_test,
                               search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
    result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'],
                                                orient='index',
                                                columns=[f'{json.dumps((list(s.values())))} balanced {metric} score'])
    
    best_estimators[f'{i}'], best_scores[f'{i}'] = \
        utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
                               search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
    result_df = pd.DataFrame.from_dict(best_scores[f'{i}'],
                                       orient='index',
                                       columns=[f'{json.dumps((list(s.values())))} {metric} score'])
    
    full_results_df = pd.concat([full_results_df, result_df_balanced, result_df], axis=1)


0 {'method': 'random'}
1 {'method': 'smote'}
2 {'method': 'cf_random', 'kw_args': {'posthoc_sparsity_param': 0.1}}
model for cf f1 score: {'cf': 0.8291746641074856}
0/272


100%|██████████| 1/1 [00:00<00:00, 11.50it/s]


50/272


100%|██████████| 1/1 [00:00<00:00, 11.55it/s]


100/272


...

100%|██████████| 1/1 [00:00<00:00,  6.10it/s]


150/221


100%|██████████| 1/1 [00:00<00:00, 15.23it/s]


200/221


100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


In [12]:
full_results_df

,full data f1 score,"[""random""] balanced f1 score","[""random""] f1 score","[""smote""] balanced f1 score","[""smote""] f1 score","[""cf_random"", {""posthoc_sparsity_param"": 0.1, ""total_CFs"": 1, ""random_seed"": 42, ""desired_class"": 0}] balanced f1 score","[""cf_random"", {""posthoc_sparsity_param"": 0.1, ""total_CFs"": 1, ""random_seed"": 42, ""desired_class"": 0}] f1 score","[""cf_random"", {""posthoc_sparsity_param"": 1, ""total_CFs"": 1, ""random_seed"": 42, ""desired_class"": 0}] balanced f1 score","[""cf_random"", {""posthoc_sparsity_param"": 1, ""total_CFs"": 1, ""random_seed"": 42, ""desired_class"": 0}] f1 score","[""cf_genetic"", {""proximity_weight"": 0.2, ""diversity_weight"": 5, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 0}] balanced f1 score",...,"[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 0}] balanced f1 score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 0}] f1 score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 0}] balanced f1 score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 0}] f1 score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 0}] balanced f1 score","[""cf_genetic"", {""proximity_weight"": 1, ""diversity_weight"": 1, ""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 0}] f1 score","[""cf_kdtree"", {""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 0}] balanced f1 score","[""cf_kdtree"", {""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 0}] f1 score","[""cf_kdtree"", {""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 0}] balanced f1 score","[""cf_kdtree"", {""sparsity_weight"": 1, ""total_CFs"": 1, ""desired_class"": 0}] f1 score"
lg,0.823529,0.759259,0.828125,0.785219,0.820000,0.799117,0.810458,0.799117,0.810458,0.787611,...,0.790287,0.792123,0.780488,0.786813,0.790287,0.793028,0.790287,0.796499,0.790287,0.796499
rf,0.822669,0.775414,0.822018,0.791946,0.818692,0.773148,0.777273,0.773148,0.777273,0.775229,...,0.782609,0.781818,0.781818,0.791946,0.790068,0.779279,0.796380,0.800885,0.796380,0.800885
xgb,0.815109,0.788546,0.816000,0.812362,0.816495,0.808696,0.810345,0.808696,0.810345,0.799136,...,0.800000,0.804348,0.802575,0.804301,0.806941,0.800857,0.800000,0.800857,0.800000,0.800857


In [13]:
full_results_df.to_csv(rf'log/experiment_{metric}_testsize{test_size_proportion}_augmentsample{augment_sample}.csv')

0.33